In [11]:
import examples, core 
import os
import yaml
import dspy

data = examples.load_examples("examples.json")
explingo = core.Explingo(context="The model predicts house prices", 
                         examples=data)

In [12]:
with open(os.path.join("..", "keys.yaml"), "r") as file:
    config = yaml.safe_load(file)
    openai_api_key = config["openai_api_key"]

llm = dspy.OpenAI(model='gpt-4', api_key=openai_api_key)
dspy.settings.configure(lm=llm, experimental=True)

# BASIC PROMPTING

TODO: update Explingo's prompting method to take in multiple different prompt options. 
DSPy does not support this by default, but we are using DSPy's evaluatiosn

In [13]:
test_explanations = [d.explanation for d in data]
test_explanation_format = data[0].explanation_format

explingo.run_experiment(test_explanations, test_explanation_format, type="basic", max_iters=5)

0.72

In [14]:
explingo.run_experiment(test_explanations, test_explanation_format, type="few-shot", max_iters=5)

0.72

In [15]:
explingo.run_experiment(test_explanations, test_explanation_format, type="bootstrap-few-shot", max_iters=5)


100%|██████████| 25/25 [00:00<00:00, 399.91it/s]


Bootstrapped 0 full traces after 25 examples in round 0.


0.72